<a href="https://colab.research.google.com/github/A00827038/Data_Structures/blob/main/M1_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad 1

En esta actividad modelaremos el comportamiento de un agentes reactivo simple–robots de limpieza.


## Reglas

Dado
1. Habitación de MxN espacios.
2. Número de agentes.
3. Porcentaje de celdas inicialmente sucias.
4. Tiempo máximo de ejecución.

Realizar lo Siguiente
1. Inicializar las celdas sucias (ubicaciones aleatorias).
2. Todos los agentes empiezan en la celda [1,1].
3. Los agentes recorreran el grid y limpiar las celdas sucias. 

*En cada paso de tiempo:
1. Si la celda está sucia, entonces aspira.
2. Si la celda está limpia, el agente elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas) 

## Imports

Antes de empezar a crear el modelo del juego de la vida con multiagentes es necesario tener instalado los siguientes paquetes:
- `python`: asegúrense de usar la versión 3+.
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`: es una biblioteca de Python para el manejo de matrices, arreglos, manipulación matemática, lógica y mucho más.
- `matplotlib`: es una biblioteca para crear visualizaciones estáticas, animadas e interactivas en Python.

Para poder modelar el juego de la vida usando el framework de `mesa` es necesario importar dos clases: una para el modelo general, y otro para los agentes. 

In [ ]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 3.4 MB/s 
     |████████████████████████████████| 60 kB 5.6 MB/s 


In [ ]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import SingleGrid #Utilizar multi-grid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## Crear el modelo

Antes que nada el presente modelo se encuentra basado en el [tutorial introductorio](https://mesa.readthedocs.io/en/master/tutorials/intro_tutorial.html). Lo modifiqué un poco para que funcionara para el presente problema pero en esencia es lo mismo.

In [ ]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente. 
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        grid[x][y] = cell_content.clean
    return grid

class robotAgent(Agent):
    '''
    Representa a un robot
    '''
    def __init__(self, unique_id, model):
        '''
        Crea un agente 
        '''
        super().__init__(unique_id, model)
    
    def step(self):
        '''
        Este método es el que mueve el robot a una diferente celda
        '''
    
    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)



class floorAgent(Agent):
    '''
    Representa a una celda con estado limpio (1) o sucio (0)
    '''
    def __init__(self, unique_id, model):
        '''
        Crea un agente con estado inicial aleatorio de 0 o 1, también se le asigna un identificador 
        formado por una tupla (x,y). También se define un nuevo estado cuyo valor será definido por las 
        reglas mencionadas arriba.
        '''
        super().__init__(unique_id, model)
        self.clean = np.random.choice([0,1])
        self.next_state = None
    
    def step(self):
        '''
        Este método es el que calcula si la celda se limpiará dependiendo si un agente robot se encuentra ahí.
        El estado live de la siguiente generación no se cambia aquí se almacena en self.next_state. La idea 
        es esperar a que todos los agentes calculen su estado y una vez hecho eso, ya hacer el cambio.
        '''
        robots = 0   
        
        #Obtiene los agentes dentro de la celda
        agents = self.model.grid.get_neighbors( 
            self.pos,
            moore=False,
            include_center=True)
        
        #Obtiene los robots dentro de la celda
        for neighbor in neighbours:
            if(neighbor.unique_id.tipo == 1){
                robots = robots + neighbor
            }
            
        
        #Si el espacio está sucio, entonces checa si hay robots
        #Si hay robots, entonces la celda se cambia a limpio
        #Si el espacio ya está limpio, entonces el agente se mueve
        self.next_state = self.clean
        if self.next_state == 0:
            if robots != 0:
                self.next_state = 1
        else:
            
    
    def advance(self):
        '''
        Define el nuevo estado calculado del método step.
        '''
        self.clean = self.next_state
            
class CleanFloorModel(Model):
    '''
    Define el modelo del juego de la vida.
    '''
    def __init__(self, N, Porcentaje, TiempoMax, width, height):
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        
        #Creamos los agentes
        for (content, x, y) in self.grid.coord_iter():
            a = robotAgent((x, y), self)
            b = floorAgent((x, y), self)
            self.grid.place_agent(a, (1, 1, t=1))
            self.grid.place_agent(b, (x, y, t=0))
            self.schedule.add(a)
            self.schedule.add(b)
        
        # Aquí definimos con colector para obtener el grid completo. Aquí recompilamos la información.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
    
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.datacollector.collect(self)
        self.schedule.step()

A continuación corremos el modelo

In [ ]:
# Definimos el tamaño del Grid
GRID_SIZE = 10

# Definimos el número de generaciones a correr
NUM_GENERATIONS = 50

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = GameLifeModel(GRID_SIZE, GRID_SIZE)
for i in range(NUM_GENERATIONS):
    model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Tiempo de ejecución: 0:00:00.048537


Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información.

In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()

Graficamos la información usando `matplotlib`

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [ ]:
anim

## CUDA

El paso siguiente será modificar el modelo anterior para que funcione con CUDA

#Conclusiones